### Chapter 7. Extending the OLG model
Reference: Fehr, H. and Kindermann, F. 2018. The overlapping generations model. Introduction to Computational Economics Using Fortran. Oxford University Press. pp253-286.

In [1]:
##### Program 7.4 Computation of marginal consumption with lifespan uncertainty

import numpy as np

# Parameters
"""
    T: transition path periods
    J: age
    JR: the year household retires
    γ: risk aversion
    egam: denominator of utility function
    β: discount factor
    ν: leisure preference parameter
    ρ: intratemporal elasticity of substitution between consumption and leisure
    eρ: denominator of leisure preference
    α: capital share
    δ: depreciation
    tol: tolerance
    damp: speed of convergence
    itermax: maximum iteration number
    ξ: parameter of education function ξe^υ
    υ: parameter of education function ξe^υ
    ϵ: the strength o the externality in Y=(K^α)L^(1-α)H^ϵ
    μ: shadow price
    ψ: survival probability
    ----------------------------------------------------------------------------
    w: wage rate
    r: rate of capital return
    wn: after tax wage (net wage)
    winc: wage income
    Rn: after tax capital return
    p: consumption price
    K: aggregate capital
    L: aggregate labour
    A: aggregate assets
    C: aggregate consumption
    I: aggregate investment
    B: aggregate debt
    G: government spending
    k: capital per capita
    a: savings
    e: education investment
    c: consumption
    U: utility
    l: leisure
    h: household productivity (human capital)
    beq: bequest level
    ω_b: weight for the bequest distribution 
    Y: aggregate output
    Ba: debt or assets of the agency per capita of workers
    Bf: future period debt B_{t+1}
    Tb: total debt (1+np)B_{t+1}+(1+r)B_t
    TXr: total tax revenue
    Tpen: total pension
    H: the average stock of human capitak per worker
    GAM: bequest distribution
    BQ: aggregate savings of the deceased
"""


def initialize():
    global T, J, JR, γ, egam, β, α, δ, tol, damp, itermax, w, r, wn, Rn, p, K, \
    L, A, C, I, B, G, k, U, Y, Ba, Bf, h, Tb, TXr, Tpen, eps, n_p, gy, by, κ, lsra_on, \
    smopec, τk, τc, τw, τr, τp, τs, tax, pen, a, c, v, m, ψ, beq, ω_b, GAM, BQ

    # Parameters
    T = 24; J = 3; JR = 3; γ = .5; egam = 1.0 - 1.0/γ; β = .9
    α = .3; δ = .0; tol = 1e-6; damp = 0.2; itermax = 200

    # Variables
    w  = np.zeros(T+1); r = np.zeros(T+1); wn = np.zeros(T+1)
    Rn = np.zeros(T+1); p = np.zeros(T+1); K  = np.zeros(T+1)
    L  = np.zeros(T+1); A = np.zeros(T+1); C  = np.zeros(T+1)
    I  = np.zeros(T+1); B = np.zeros(T+1); G  = np.zeros(T+1)
    k  = np.zeros(T+1); U = np.zeros((J,T+1)); Y = np.zeros(T+1)
    Ba = np.zeros(T+1); Bf = np.zeros(T+1); h = np.zeros(J)
    Tb = np.zeros(T+1); TXr = np.zeros(T+1); Tpen = np.zeros(T+1)
    ψ = np.zeros((J,T+1)); beq = np.zeros((J,T+1)); ω_b = np.zeros(J)
    GAM = np.zeros((J,T+1)); BQ = np.zeros(T+1)

    # Set model parameters
    n_p = np.zeros(T+1) + .2    #population growth rate
    n_e = np.zeros(T+1)         #endogenous growth rate in human capital (productivity)
    gy  = 0.2                   #government spending proportion
    by  = np.zeros(T+1)         #debt proportion
    κ   = np.zeros(T+1)         #pension proportion
    lsra_on = False             #include welfare analysis or not
    smopec = False              #include small open economy or not
    
    # Set the survival probabilities
    ψ[0,:] = 1.0
    ψ[1,:] = 0.85
    ψ[2,:] = 0.8

    # Initialize tax rates shadow wages and pensions
    τk  = np.zeros(T+1)         #investment tax rate
    τc = np.zeros(T+1)          #consumption tax rate
    τw = np.zeros(T+1)          #labour income tax rate
    τr = np.zeros(T+1)          #capital income tax rate
    τp = np.zeros(T+1)          #payroll tax rate
    τs = np.zeros(T+1)          #subsidy rate of the government for education costs
    tax = np.ones(T+1)          #tax balancing index
    pen = np.zeros((J,T+1))     #pension

    # Initialize assets
    a = np.zeros((J,T+1))       #assets
    c = np.zeros((J,T+1))       #consumption
    v = np.zeros(T+J-1)         #reqiured transfer
    
    h[0:JR-1] = 1.0             #human capital (probability)
    h[JR-1:J] = .0              #no human capital accumulation after retirement
    
    # Set bequest distribution
    ω_b[0] = .5
    ω_b[1] = .5
    ω_b[2] = .0

    # Size of cohorts and bequest distribution in specific year
    m = np.zeros((J,T+1))       #population parameter
    for t in range(T+1):
        m[0,t] = 1.0            #in period 0, m=1
        tm = year(t, 1, 0)      #time index t-1
        for j in range(1,J):
            m[j,t] = m[j-1,tm]*ψ[j,t]/(1.0 + n_p[t]) #m_{j,t}=m_{j-1,t-1}ψ_{j,t}/(1+np)
            
        GAM_total = ω_b[0]      #initial weight of bequest distribution (age 0)
        for j in range(1,J):
            GAM_total = GAM_total + ω_b[j]*m[j,t]  #total value of weighted bequest
            
        for j in range(J):
            GAM[j,t] = ω_b[j]/GAM_total            #the proportion of bequest in each age cohort
    return None


In [2]:
## calculates year at which age j agent is jp
def year(t, j, jp):
    """This function defines the year index"""
    year = t + jp - j  #year i age j period ij (j-1, j+1, j+2)
    if t == 0 or year<=0:
        year = 0       #initial period
    if t == T or year>=T:
        year = T       #last period
    return year

In [3]:
## Function: factor price:
def factor_prices(t):
    """This function defines multidimensional factor price calculation for period i."""
    global k, r, w, wn, Rn, p
    k[t]   = K[t]/L[t]                  #per capita capital stock
    if smopec and t > 0:
        r[t] = r[0]
    else:
        r[t] = (1.0 - τk[t])*(α*k[t]**(α-1.0)-δ)  #(1-τk)*r
  
    w[t]   = (1.0 - α)*k[t]**α  #wage rate

    wn[t]  = w[t]*(1.0 - τw[t] - τp[t])    #after tax wage rate
    Rn[t]  = 1.0 + r[t]*(1.0 - τr[t])      #net interest
    p[t]   = 1.0 + τc[t]                   #consumer price
    return None

In [4]:
## present value of resources for household aged j in year t

def get_W(j, t):
    """This function calculates the total income at each age j at period t."""
    # get current value of resources
    Assets = wn[t]*h[j] + beq[j,t] + pen[j,t]       #initial total income include the income, bequest, and pension
    
    if t == 1 and j > 0:                            #if period 1 and age 1,2,...
        ## calful about index!!!
        Assets = Assets + Rn[t] * a[j,t] + v[J - j - 1] #initial assets + capital return + required transfer 
    if t >= 1 and j == 0:                           #if age 0
        Assets = Assets + v[t+J-2]                  #initial assets + required transfer
        
    # Iterate over remainder of life span
    PRn = 1.0
    for jp in range(j+1,J):  #loop for remainder of life span
        tp = year(t,j,jp)    #time index for remainder of life span
        PRn = PRn*Rn[tp]     #cumulated return on capital Rn[t+1] and Rn[t+1]*Rn[t+2]
        Assets = Assets + (wn[tp]*h[jp] + beq[jp,tp] + pen[jp,tp])/PRn   #W equation above (7.25)
    return Assets

## marginal consumption for household aged j in year t

def get_Psi(j,t):
    """This function calculates the consumption proportion at each age j at period t."""
    Psi = 1.0         #set initial value of Psi
    PRn = 1.0         #PRn in the period 0 is 1
    PPs = 1.0         #unconditional survival probability in period 0 is 1
    
    for jp in range(j+1,J):  #loop for each future ages
        tp = year(t, j ,jp)  #year interval t+jp-j
        PRn = PRn*Rn[tp]     #cumulated return on capital Rn[t+1] and Rn[t+1]*Rn[t+2]
        PPs = PPs*ψ[jp,tp]   #unconditional survival probability
        Psi = Psi + (β**(jp-j)*PPs)**γ*(p[tp]/(PRn*p[t]))**(1.0-γ)  #part inside big bracket of (7.28)
    Psi = 1.0/(p[t]*Psi)     #1/p/(big bracket) equation (7.25)
    
    return Psi


In [5]:
## Calculating the optimal consumption path

def get_path(j,t):
    """This function calculates the optimal consumption path at each age j at period t."""
    global c, a, beq
    
    PRn = 1.0         #PRn in the period 0 is 1
    beq[j,t] = GAM[j,t]*BQ[t]  #bequest is distributed according to the bequest distribution for each cohort

    for jp in range(j+1,J):  #loop for future ages (j+1,...,J)
        # get future and previous year as well as interest factor
        tp = year(t,j,jp)    #t+jp-j
        tm = year(t,j,jp-1)  #t+jp-j-1
        PRn = PRn*ψ[jp,tp]*Rn[tp]     #survived cumulated return on capital Rn[t+1] and Rn[t+1]*Rn[t+2]
        
        # get consumption and leisure
        c[jp,tp] = (β**(jp-j)*PRn*p[t]/p[tp])**γ*c[j,t]       #c_{2,t+1} and c_{3,t+2} equation (7.13&14)
        beq[jp,tp] = GAM[jp,tp]*BQ[tp] #bequest is distributed according to the bequest distribution for each cohort
        # calculate assets 
        a[jp,tp] = wn[tm]*h[jp-1] + beq[jp-1,tm] + pen[jp-1,tm] + Rn[tm]*a[jp-1,tm] - p[tm]*c[jp-1,tm]
        #savings as all income (labour income + bequest + pension + capital income) - consumption
        if tp == 2:
            a[jp,tp] = a[jp,tp] + v[J-jp]   #a + required transfer if include welfare analysis
        if tp > 2 and jp == 1:
            a[jp,tp] = a[jp,tp] + v[tm+J-2] #a + required transfer if include welfare analysis
    return None


In [6]:
## Calculating individual decisions in a certain year

def decisions(t):
    """This function defines the decisions of consumption and savings in each period t."""
    global c
    
    c[0,t] = get_Psi(0,t)*get_W(0,t) #c=ΨW
    get_path(0,t)                    #calculate consumption path at age 0 period t
    
    if t == 1:
        for j in range(1,J):
            c[j,t] = get_Psi(j,t)*get_W(j,t)
            get_path(j,t)
    return None

In [7]:
## calculating quantities in a certain year

def quantities(t):
    """This function defines the aggregated quantities of C, A, Y, I, G, B, K, Bf, Tb, I, BQ in period t."""
    global G, C, A, L, Y, B, K, Bf, Tb, I, BQ
    
    tm = year(t,1,0)   #time index t-1
    tp = year(t,0,1)   #time index t+1
    
    if t == 0:         #initial goverment spending
        G[t] = gy*Y[t] #goverment spending
    else:              #other periods
        G[t] = G[0]    #same as period 0
        
    # aggregate individual decisions
    C[t] = .0          #initial value of aggregate consumption
    A[t] = .0          #initial value of aggregate assets
    L[t] = .0          #initial value of aggregate labour input
    BQ[t] = .0         #initial value of aggregate bequest
    
    for j in range(J): #for all ages (equations on page 276)
        C[t] = C[t] + c[j,t]*m[j,t]  #cumulated aggregate consumption with population parameters mt=m_{t-1}/(1+np)
        A[t] = A[t] + a[j,t]*m[j,t]/ψ[j,t] #cumulated aggregate assets with population parameters mt/ψ
        L[t] = L[t] + h[j]*m[j,t]    #cumulated aggregate labour with population parameters mt=m_{t-1}/(1+np)
        BQ[t] = BQ[t] + (1.0 - ψ[j,t])*Rn[t]*a[j,t]*m[j,t]/ψ[j,t] #cumulated aggregate bequest with population parameters mt/ψ
    
    Y[t] = K[t]**α*L[t]**(1.0-α)  #aggregate output
    B[t] = by[tm]*Y[t]            #aggregate debt
    
    # derive capital in small open or closed economy
    if smopec and t>0:
        K[t]  = L[t]*((r[t]/(1.0-τk[t]) + δ)/α)**(1.0/(α-1.0))  #converted from r=α(K/L)^(α-1)-δ 
        Bf[t] = A[t] - K[t] - Ba[t] - B[t]               #debt constraint
        Tb[t] = (1.0+n_p[tp])*Bf[tp] - (1.0+r[t])*Bf[t]  #total debt
    else:
        K[t] = damp*(A[t]-B[t]-Ba[t]) + (1.0-damp)*K[t]  #converging capital stock
    
    I[t] = (1.0+n_p[tp])*K[tp] - (1.0 - δ)*K[t]          #aggregate investment
    
    return None


In [8]:
# calculating government parameters

def government(t):
    """This function defines the government parameters and tax revenues at period t."""
    global τc, τk, τw, τr, pen, Tpen, τp, TXr
    tp = year(t, 0, 1)             #t+1
    
    taxrev = np.zeros(4)           #set 4 types of taxes
    
    taxrev[0] = τc[t]*C[t]         #total consumption tax
    taxrev[1] = τw[t]*w[t]*L[t]    #total labour income tax
    taxrev[2] = τr[t]*r[t]*A[t]    #total capital income tax (on interest return)
    taxrev[3] = τk[t]*(Y[t] - w[t]*L[t]-(δ)*K[t])  #total savings or investment tax
    
    # get budget balancing tax rate
    if tax[t] == 1:      #(government surplus-τcC)/C
        τc[t] = ((1.0 + r[t])*B[t] + G[t] - (taxrev[1] + taxrev[2] + \
                    taxrev[3]+(1.0 + n_p[tp])*B[tp]))/C[t]   
    elif tax[t] == 2:    #[government surplus-τw(wL)-τr(rA)]/(wL+rA) assuming same labour and capital income tax
        τw[t] = ((1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[3] + \
                    (1.0+n_p[tp])*B[tp]))/(w[t]*L[t] + r[t]*A[t])
        τr[t] = τw[t]    #same labour and capital income tax
    elif tax[t] == 3:    #[government surplus-τw(WL)]/(wL)
        τw[t] = ((1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[2] + \
                    taxrev[3] + (1.0 + n_p[tp])*B[tp]))/(w[t]*L[t])
    else:                #[government surplus-τr(rA)]/(rA)
        τr[t] = ((1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[1] + \
                    taxrev[3]+ (1.0 + n_p[tp])*B[tp]))/(r[t]*A[t])
    
    TXr[t] = np.sum(taxrev)   #total tax revenue
    
    # get budget balancing social security contribution
    pen[JR-1:J,t] = κ[t]*w[t] #pension at each period
    Tpen[t] = .0              #initial total pension
    for j in range(JR-1,J):   #for retired generation
        Tpen[t] = Tpen[t] + pen[j,t]*m[j,t] #total pension with population parameters mt=m_{t-1}/(1+np)
        
    τp[t] = Tpen[t]/(w[t]*L[t]) #payroll tax rate
    return None

In [9]:
# computing life expectancy of household who enters in year t

def ET(t):
    """This function computes the life expectancy at period t."""
    ET = .0            #initial liffe expectancy
    PPs = 1.0          #initial unconditional survival probability
    
    for j in range(J): #for each period
        tp = year(t, 0, j)    #t+j
        tp1 = year(t, 0, j+1) #t+j+1
        PPs = PPs*ψ[j, tp]    #calculating unconditional survival probability
    
        if j<J-1:             #before last period
            ET = ET + (j+1)*PPs*(1.0-ψ[j+1, tp1])  #rising life expectancy equation on page 279
        else:
            ET = ET + (j+1)*PPs                    #in the last period, ψ=0
    return ET


In [10]:
## Compute household utility

def utility(t):
    """This function defines expected utility at each age for each period i in [0,T]."""
    global U
    
    # for first generation
    U[0,t] = .0
    PPs = 1.0
    for j in range(J):
        tp = year(t, 0, j)
        PPs = PPs*ψ[j,tp]
        U[0,t] = U[0,t] + β**j*PPs*c[j,tp]**egam/egam
    
    # for current total population if year = 0 or 1
    if t < 2:
        for j in range(1,J):
            U[j,t] = c[j,t]**egam/egam
            PPs = 1.0
            for jp in range(j+1,J):
                tp = year(t, j, jp)
                PPs = PPs*ψ[jp,tp]
                U[j,t] = U[j,t] + β**(jp-j)*PPs*c[jp,tp]**egam/egam
                
    return None

In [11]:
## Calculating lsra transfers

def lsra():
    """This function calculates social welfare. True to include, False to exclude."""
    global Ba, v, ustar

    #calculate utility for each generation
    for t in range(1,T+1):
        utility(t)   #get utility for each period i=0,1,2,...T
        
    Ba = np.zeros(T+1)  #initialize debt
    for j in range(1,J):
        v[J-j-1] = v[J-j-1] + get_W(j,1)*((U[j,0]/U[j,1])**(1.0/egam)-1.0)
        #v: required transfer that bring older cohorts in the reform year from their current utility level 
        #back to their initial equilibrium utilities
        Ba[2] = Ba[2] + v[J-j-1]*m[j,1]  #debt or assets of the agency per capita of workers
    
    #Calculate the following separated terms of the equation to solve for U^*.
    # long run equilibrium (last period) 
    PV = v[T+J-2]*(1.0+r[T])/(r[T]-n_p[T])                                 #v/(r-n_p) in period T part
    sum1 = get_W(0,T)*(1.0+r[T])/(r[T]-n_p[T])                             #w/(r-n_p) in period T part
    sum2 = get_W(0,T)*(U[0,T]*egam)**(-1.0/egam)*(1.0+r[T])/(r[T]-n_p[T])  #normalized W/Ut in period T part
    
    # transition path (backward induction)
    for t in range(T-1,0,-1): # from T to 1.
        tp = year(t,0,1)      #t+1
        PV = PV*(1.0+n_p[tp])/(1.0+r[tp]) + v[t+J-2]                       #v(1+np)/(1+r) part
        sum1 = sum1*(1.0+n_p[tp])/(1.0+r[tp]) + get_W(0,t)                 #W(1+np)/(1+r) part
        sum2 = sum2*(1.0+n_p[tp])/(1.0+r[tp]) + get_W(0,t)*(U[0,t]*egam)**(-1.0/egam) # normalized W/Ut part
        
    # calculate ustar for future generations 
    ustar = ((sum1 - Ba[2] - PV)/sum2)**egam/egam   #the identical utility level for future generations
    
    # calculate transfers to future generations and debt of LSRA
    for t in range(1,T+1):
        v[t+J-2] = v[t+J-2] + get_W(0,t)*((ustar/U[0,t])**(1.0/egam)-1.0)  #lump-sum transfers for future cohorts
        if t == 2:
            Ba[2] = (Ba[2] + v[J-1])/(1.0 + n_p[2])               #B_{t+1}=(above Ba+v)/(1+np) equation on page 243
        if t > 2:
            Ba[t] = ((1.0+r[t-1])*Ba[t-1] + v[J+t-3])/(1.0+n_p[t])#B_{t+1}=((1+r)Ba+v)/(1+np) same equation on page 243
    
    return None

In [12]:
# solves inital steady state using Gauss-Seidel

def get_SteadyState():
    """This function calculates inital steady state using Gauss-Seidel."""
    K[0] = 1.0  #initial capital
    L[0] = 1.0  #initial labour
    for i in range(itermax+1):
        factor_prices(0)
        decisions(0)
        quantities(0)
        government(0)
        
        if abs(Y[0]-C[0]-I[0]-G[0])/Y[0] < tol:
            break
    utility(0)
    
    if i < itermax:
        print('Iteration: ', i, ' Diff: ',abs(Y[0]-C[0]-I[0]-G[0])/Y[0])
    else:
        print('!!! No equilibrium found !!!')
    
    return None

In [13]:
## sovles for transition path using Gauss-Seidel:

def get_Transition():
    """This function calculates each period economy and checks for market clearing conditions."""
    global a, K, L, HEV
    # Initialize values from initial equilibrium
    a[:,1] = a[:,0]  #savings
    K[:] = K[0]      #aggregate capital stock
    L[:] = L[0]      #aggregate labour
    
    nmarket = 0      #initialize iteration for number of markets cleared
    
    for ite in range(itermax+2):
        # get prices, decisions and quantites
        
        for t in range(1,T+1):
            factor_prices(t)
        
        for t in range(1,T+1):
            decisions(t)
        
        if lsra_on:
            lsra()
        
        for t in range(1,T+1):
            quantities(t)
        
        for t in range(1,T+1):
            government(t)
            
        # check for the number of markets in equilibrium
        nmarket = 0
        for t in range(1,T+1):
            if  abs(Y[t]-C[t]-I[t]-G[t]-Tb[t])/Y[t] < tol:
                nmarket = nmarket + 1
        if nmarket == T:
            break
        print('Iteration: ',ite,' Markets: ',nmarket,' Diff: ',np.max(np.abs(Y-C-I-G-Tb)/Y))
    if ite < itermax:
        print('Iteration: ',ite,' Markets: ',nmarket,' Diff: ',np.max(np.abs(Y-C-I-G-Tb)/Y))
    else:
        print('!!! No equilibrium found !!!')
        
        
    for t in range(1,T+1):
        utility(t)
    HEV = np.zeros(T+2)
    for t in range(2,T+2):
        HEV[t] = ((U[0,t-1]/U[0,0])**(1/egam)-1)*100
    HEV[1] = ((U[1,1]/U[1,0])**(1/egam)-1)*100 # generation 0
    HEV[0] = ((U[2,1]/U[2,0])**(1/egam)-1)*100 # generation -1
        
    return None

In [14]:
# Table 7.11
initialize()
κ[1:] = .5 # gy = .2
get_SteadyState()
get_Transition()
print('---------------------------------------------------')
print('Table 7.11 Introducing pay-as-you-go pensions with\nuncertain life span')
print('---------------------------------------------------')
print(' t   κ   τp    τc    C   BQ    K    w    r   HEV')
print('---------------------------------------------------')
print(-1,'                                       ','%2.2f'%HEV[0])
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%κ[t],'%.2f'%τp[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%BQ[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
for t in [T]:
    print('%2.f'%t,'%.2f'%κ[t],'%.2f'%τp[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%BQ[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t],'%2.2f'%HEV[t+1])
print('---------------------------------------------------')


Iteration:  83  Diff:  9.366477935986829e-07
Iteration:  0  Markets:  0  Diff:  0.12974942109060705
Iteration:  1  Markets:  0  Diff:  0.1262596091804003
Iteration:  2  Markets:  0  Diff:  0.07845613982561954
Iteration:  3  Markets:  0  Diff:  0.06347269398995758
Iteration:  4  Markets:  0  Diff:  0.07202901881531494
Iteration:  5  Markets:  0  Diff:  0.07537717467103364
Iteration:  6  Markets:  0  Diff:  0.08514076405021098
Iteration:  7  Markets:  0  Diff:  0.06872770924135563
Iteration:  8  Markets:  0  Diff:  0.04117982226098736
Iteration:  9  Markets:  0  Diff:  0.03362760021846013
Iteration:  10  Markets:  0  Diff:  0.03385062004701189
Iteration:  11  Markets:  0  Diff:  0.04334010709195395
Iteration:  12  Markets:  0  Diff:  0.04888393606819715
Iteration:  13  Markets:  0  Diff:  0.04304058400197231
Iteration:  14  Markets:  0  Diff:  0.02999475656633514
Iteration:  15  Markets:  0  Diff:  0.024596687115462847
Iteration:  16  Markets:  0  Diff:  0.026521576978388965
Iteration:  

In [15]:
print(ψ)

[[1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.  ]
 [0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85
  0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85 0.85]
 [0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8
  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8 ]]


In [16]:
# Table 7.12
initialize()
ψ[1,2] = .90001; ψ[1,3:] =.950001; ψ[2,3:5] = 0.850001; ψ[2,5:] = .9000001
#ψ[1,:] = .850001; ψ[2,:] = .800001
gy = .0; β = .85
get_SteadyState()
get_Transition()
print('---------------------------------------------------')
print('Table 7.12 Macroeconomics of rising life expectancy')
print('---------------------------------------------------')
print(' t  ψ2t  ψ3t E[T]  C    L    BQ   K    w    r')
print('---------------------------------------------------')
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%ψ[1,t],'%.2f'%ψ[2,t],'%.2f'%ET(t),'%.2f'%C[t],'%.2f'%L[t],'%.2f'%BQ[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
for t in [T]:
    print('%2.f'%t,'%.2f'%ψ[1,t],'%.2f'%ψ[2,t],'%.2f'%ET(t),'%.2f'%C[t],'%.2f'%L[t],'%.2f'%BQ[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print('---------------------------------------------------')


Iteration:  83  Diff:  9.580017343748177e-07
Iteration:  0  Markets:  0  Diff:  0.10760312832875533
Iteration:  1  Markets:  0  Diff:  0.12635087445859294
Iteration:  2  Markets:  0  Diff:  0.06014648297532832
Iteration:  3  Markets:  0  Diff:  0.04627227659933837
Iteration:  4  Markets:  0  Diff:  0.04762625293953297
Iteration:  5  Markets:  0  Diff:  0.04727102428763135
Iteration:  6  Markets:  0  Diff:  0.04539327861211
Iteration:  7  Markets:  0  Diff:  0.04269481893697519
Iteration:  8  Markets:  0  Diff:  0.04097072363250589
Iteration:  9  Markets:  0  Diff:  0.03976555044855448
Iteration:  10  Markets:  0  Diff:  0.03873702514136772
Iteration:  11  Markets:  0  Diff:  0.037760019644982844
Iteration:  12  Markets:  0  Diff:  0.03689389848117677
Iteration:  13  Markets:  0  Diff:  0.03616043290669946
Iteration:  14  Markets:  0  Diff:  0.0355373817797283
Iteration:  15  Markets:  0  Diff:  0.03500021507020134
Iteration:  16  Markets:  0  Diff:  0.03453556065832596
Iteration:  17  

Iteration:  153  Markets:  0  Diff:  0.0327950900430711
Iteration:  154  Markets:  0  Diff:  0.03279509004307174
Iteration:  155  Markets:  0  Diff:  0.032795090043072454
Iteration:  156  Markets:  0  Diff:  0.0327950900430729
Iteration:  157  Markets:  0  Diff:  0.03279509004307341
Iteration:  158  Markets:  0  Diff:  0.03279509004307351
Iteration:  159  Markets:  0  Diff:  0.03279509004307411
Iteration:  160  Markets:  0  Diff:  0.03279509004307428
Iteration:  161  Markets:  0  Diff:  0.03279509004307405
Iteration:  162  Markets:  0  Diff:  0.032795090043074585
Iteration:  163  Markets:  0  Diff:  0.032795090043074716
Iteration:  164  Markets:  0  Diff:  0.03279509004307467
Iteration:  165  Markets:  0  Diff:  0.032795090043075036
Iteration:  166  Markets:  0  Diff:  0.03279509004307471
Iteration:  167  Markets:  0  Diff:  0.03279509004307529
Iteration:  168  Markets:  0  Diff:  0.03279509004307521
Iteration:  169  Markets:  0  Diff:  0.03279509004307521
Iteration:  170  Markets:  0 

In [17]:
print(ψ)

[[1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.       ]
 [0.85      0.85      0.90001   0.950001  0.950001  0.950001  0.950001
  0.950001  0.950001  0.950001  0.950001  0.950001  0.950001  0.950001
  0.950001  0.950001  0.950001  0.950001  0.950001  0.950001  0.950001
  0.950001  0.950001  0.950001  0.950001 ]
 [0.8       0.8       0.8       0.850001  0.850001  0.9000001 0.9000001
  0.9000001 0.9000001 0.9000001 0.9000001 0.9000001 0.9000001 0.9000001
  0.9000001 0.9000001 0.9000001 0.9000001 0.9000001 0.9000001 0.9000001
  0.9000001 0.9000001 0.9000001 0.9000001]]
